# HPH第二版实验

### 树的叶子节点作为特征

参考: https://github.com/lytforgood/MachineLearningTrick/blob/master/xgboost%E8%B0%83%E5%8F%82%E6%BC%94%E7%A4%BA.md

*改造cv函数, 把按天分割数据封装成DateTimeSplit类*

*GridSearchCV调整参数*


In [ ]:
import sys
sys.path.append("..")
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline
offline_df = pd.read_table('../../round1_ijcai_18_train_20180301.txt', sep=' ')
# online_df = pd.read_table('../../round1_ijcai_18_test_a_20180301.txt', sep=' ')

# 1 数据集统一处理

In [ ]:
# 时间处理: 分离天, 星期几, 上中下午/晚上, 小时数

from datetime import datetime

def extract_date(x):
    d = datetime.fromtimestamp(x)
    return d.strftime('%Y-%m-%d')
def extract_weekday(x):
    d = datetime.fromtimestamp(x)
    return d.weekday()
def extract_hour(x):
    d = datetime.fromtimestamp(x)
    return d.hour

offline_df['date'] = offline_df['context_timestamp'].apply(lambda x: extract_date(x))
offline_df['weekday'] = offline_df['context_timestamp'].apply(lambda x: extract_weekday(x))
offline_df['hour'] = offline_df['context_timestamp'].apply(lambda x: extract_hour(x))

In [ ]:
%run ../util/time_utils.py

offline_df = getColDupByDate(offline_df, 'date', 'item_id', 3)
offline_df = getColDupByDate(offline_df, 'date', 'item_brand_id', 3)
offline_df = getColDupByDate(offline_df, 'date', 'shop_id', 3)
offline_df = getColDupByDate(offline_df, 'date', 'user_id', 3)

offline_df = getColDupByDate(offline_df, 'date', 'item_id', 7)
offline_df = getColDupByDate(offline_df, 'date', 'item_brand_id', 7)
offline_df = getColDupByDate(offline_df, 'date', 'shop_id', 7)
offline_df = getColDupByDate(offline_df, 'date', 'user_id', 7)
offline_df = getColDupByDate(offline_df, 'date', 'item_id', 1)
offline_df = getColDupByDate(offline_df, 'date', 'item_brand_id', 1)
offline_df = getColDupByDate(offline_df, 'date', 'shop_id', 1)
offline_df = getColDupByDate(offline_df, 'date', 'user_id', 1)


In [ ]:
# 统计信息附加

def getColTradeRate(df, idCol):
    rateCol = idCol + '_tr'
    pvCol = idCol + '_pv'
    try:
        del df[rateCol]
        del df[pvCol]
    except:
        pass
    a = offline_df.groupby([idCol]).agg({'is_trade':'sum'})
    b = offline_df.groupby([idCol]).agg({'is_trade':'size'})
    c = a.join(b, lsuffix="_sum", rsuffix="_size")
    c[rateCol] = c['is_trade_sum'] / c['is_trade_size']
    c[pvCol] = c['is_trade_size']
    return offline_df.join(c[[rateCol, pvCol]], on=idCol)
    

# 各类id的 交易量, 交易率
offline_df = getColTradeRate(offline_df, 'item_city_id')
offline_df = getColTradeRate(offline_df, 'item_id')
offline_df = getColTradeRate(offline_df, 'shop_id')
offline_df = getColTradeRate(offline_df, 'item_brand_id')
offline_df = getColTradeRate(offline_df, 'user_occupation_id')


In [ ]:
# 分桶
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

offline_df['item_city_id_pv_cut'] = pd.cut(offline_df['item_city_id_pv'].values, bins=10, retbins=False)
offline_df['item_id_pv_cut'] = pd.cut(offline_df['item_id_pv'].values, bins=10, retbins=False)
offline_df['shop_id_pv_cut'] = pd.cut(offline_df['shop_id_pv'].values, bins=10, retbins=False)
offline_df['item_brand_id_pv_cut'] = pd.cut(offline_df['item_brand_id_pv'].values, bins=10, retbins=False)
offline_df['user_occupation_id_pv_cut'] = pd.cut(offline_df['user_occupation_id_pv'].values, bins=10, retbins=False)

offline_df['item_city_id_pv_cut'] = le.fit_transform(offline_df['item_city_id_pv_cut'].copy())
offline_df['item_id_pv_cut'] = le.fit_transform(offline_df['item_id_pv_cut'].copy())
offline_df['shop_id_pv_cut'] = le.fit_transform(offline_df['shop_id_pv_cut'].copy())
offline_df['item_brand_id_pv_cut'] = le.fit_transform(offline_df['item_brand_id_pv_cut'].copy())
offline_df['user_occupation_id_pv_cut'] = le.fit_transform(offline_df['user_occupation_id_pv_cut'].copy())

In [ ]:
# standardization and scaling

from sklearn import preprocessing

def scale(df, idCol):
    min_max_scaler = preprocessing.MinMaxScaler()
    a = min_max_scaler.fit_transform(df[idCol].values.reshape(-1, 1))
    return pd.Series(a.reshape(1, -1)[0])

# 令数据再少一些?
offline_df['shop_score_service_scaled'] = scale(offline_df, 'shop_score_service')
offline_df['shop_score_delivery_scaled'] = scale(offline_df, 'shop_score_delivery')
offline_df['shop_score_description_scaled'] = scale(offline_df, 'shop_score_description')
offline_df['shop_review_positive_rate_scaled'] = scale(offline_df, 'shop_review_positive_rate')
offline_df['item_city_id_tr_scaled'] = scale(offline_df, 'item_city_id_tr')
offline_df['item_id_tr_scaled'] = scale(offline_df, 'item_id_tr')
offline_df['shop_id_tr_scaled'] = scale(offline_df, 'shop_id_tr')
offline_df['item_brand_id_tr_scaled'] = scale(offline_df, 'item_brand_id_tr')
offline_df['user_occupation_id_scaled'] = scale(offline_df, 'user_occupation_id_tr')

# offline_df = offline_df.dropna()

# 2 生成矩阵数据

In [ ]:
used = 78261
D = offline_df['context_timestamp']
X = offline_df[
    [
        'item_id', 'item_brand_id', 'item_city_id', 'user_id', 'context_page_id', 'context_id', 'shop_id', 
        'item_id_dup_g_7', 'shop_id_dup_g_7', 'item_brand_id_dup_g_7', 'user_id_dup_g_7',
#         'item_id_dup_g_1', 'shop_id_dup_g_1', 'item_brand_id_dup_g_1', 'user_id_dup_g_1',
#         'item_id_dup_g_3', 'shop_id_dup_g_3', 'item_brand_id_dup_g_3', 'user_id_dup_g_3',
        'item_id_pv_cut', 'shop_id_pv_cut', 'user_occupation_id_pv_cut', 
         'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 
        'user_gender_id', 'weekday', 'hour', 'user_age_level', 'user_star_level', 
       'user_occupation_id', 
       'item_city_id_tr', 'item_city_id_pv_cut', 'item_id_tr', 'item_id_pv',
       'shop_id_tr', 'shop_id_pv', 'item_brand_id_tr', 'item_brand_id_pv',
       'user_occupation_id_pv', 'shop_score_delivery',
        'shop_score_description', 'shop_review_positive_rate', 'shop_score_service'      
    ]
].values
#        'shop_score_delivery_scaled', 'shop_score_description_scaled',
#        'shop_review_positive_rate_scaled', 'item_city_id_tr_scaled',
#        'item_id_tr_scaled', 'shop_id_tr_scaled', 'item_brand_id_tr_scaled',k
#        'user_occupation_id_scaled', 'shop_score_service_scaled', 
        
y = offline_df[['is_trade']].values.ravel()

# 3 GridSearchCV 超参搜索框架

In [ ]:
from sklearn.model_selection import GridSearchCV as GSCV   #Perforing grid search
from sklearn.metrics import make_scorer
from sklearn.metrics import log_loss
import xgboost

%run ../util/time_series_split.py


dtsv = DateTimeSplit(dateSeries=D, fmt="%Y-%m-%d")
dtsv2 = DateTimeSplit2(dateCol='day', gap=6)

score_func = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

"""
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=20, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
"""

parameters = {
    'nthread':[20], #when use hyperthread, xgboost may become slower
    'learning_rate': [0.1], #so called `eta` value
    'colsample_bylevel': [1],
    'max_delta_step':[0],
    'colsample_bytree':[1],
    'gamma':[0],
    'max_depth': [3],
    'missing':[None],
    'min_child_weight':[1],
    'scale_pos_weight':[1],
    'n_estimators': [100],
    'base_score':[0.5],
    'booster':['gbtree'],
    'objective':['binary:logistic'],
    'reg_alpha':[0],
    'reg_lambda':[1],
    'silent':[True],
    'subsample':[1]
}

xgb = xgboost.XGBClassifier()
gs = GSCV(xgb, parameters, scoring=score_func, cv=dtsv)
# gs.fit(X, y)

print(gs.best_estimator_, gs.best_score_)

# 4 按天时间序列kfold 测试

In [77]:
D = offline_df['context_timestamp']
X = offline_df[
    [
        'item_id', 'item_brand_id', 'item_city_id', 'user_id', 'context_page_id', 'context_id', 'shop_id', 
        'item_id_dup_g_7', 'shop_id_dup_g_7', 'item_brand_id_dup_g_7', 'user_id_dup_g_7',
#         'item_id_dup_g_1', 'shop_id_dup_g_1', 'item_brand_id_dup_g_1', 'user_id_dup_g_1',
#         'item_id_dup_g_3', 'shop_id_dup_g_3', 'item_brand_id_dup_g_3', 'user_id_dup_g_3',
        'item_id_pv_cut', 'shop_id_pv_cut', 'user_occupation_id_pv_cut', 
         'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 
        'user_gender_id', 'weekday', 'hour', 'user_age_level', 'user_star_level', 
       'user_occupation_id', 
       'item_city_id_tr', 'item_city_id_pv_cut', 'item_id_tr', 'item_id_pv',
       'shop_id_tr', 'shop_id_pv', 'item_brand_id_tr', 'item_brand_id_pv',
       'user_occupation_id_pv', 'shop_score_delivery',
        'shop_score_description', 'shop_review_positive_rate', 'shop_score_service'      
    ]
].values
#        'shop_score_delivery_scaled', 'shop_score_description_scaled',
#        'shop_review_positive_rate_scaled', 'item_city_id_tr_scaled',
#        'item_id_tr_scaled', 'shop_id_tr_scaled', 'item_brand_id_tr_scaled',k
#        'user_occupation_id_scaled', 'shop_score_service_scaled', 
        
y = offline_df[['is_trade']].values.ravel()

In [82]:
##合并维度
import numpy as np
def mergeToOne(X,X2):
    X3=[]
    for i in np.arange(X.shape[0]):
        tmp=np.array([list(X[i]),list(X2[i])])
        X3.append(list(np.hstack(tmp)))
    X3=np.array(X3)
    return X3

In [84]:
# 按照普通的TimeSeriesSplit, 效果不稳定, 分数非单调提升, 感觉意义不大.

from sklearn.metrics import log_loss
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split

dtsv = DateTimeSplit(dateSeries=D, fmt="%Y-%m-%d")
# tscv = TimeSeriesSplit(n_splits=20)

# help(dtsv)
# pred_leaf=True

layer1_param = {
    'learning_rate': 0.3, #默认0.3
    'n_estimators': 30, #树的个数
    'max_depth': 3,
    'min_child_weight': 1,
    'gamma': 0.5,
    'subsample': 0.6,
    'colsample_bytree' : 0.6,
    'objective': 'binary:logistic', #逻辑回归损失函数
    'nthread': 4,  #cpu线程数
    'scale_pos_weight' : 1,
    'reg_alpha': 1e-05,
    'reg_lambda': 1,
    'seed': 27
}

layer2_param = {
    'learning_rate': 0.1,
    'n_estimators':1000,
    'max_depth':3,
    'min_child_weight':1,
    'gamma': 0.5,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'objective': 'binary:logistic', 
    'nthread': 4, 
    'scale_pos_weight': 1,
    'reg_alpha': 1e-05,
    'reg_lambda': 1,
    'seed': 27
}

for train_i, test_i in dtsv.split(X, y):
    X_train, y_train = X[train_i], y[train_i]
    X_test, y_test = X[test_i], y[test_i]
    
    X_train_1, X_train_2, y_train_1, y_train_2 = train_test_split(X_train, y_train, test_size=0.6, random_state=0)
    
    m1 = xgboost.XGBClassifier(**layer1_param)
    m1.fit(X_train_1, y_train_1)
    new_feature = m1.apply(X_train_2)
    print(new_feature)
    
    
    X_train_new2 = mergeToOne(X_train_2, new_feature)
    new_feature_test = m1.apply(X_test)
    X_test_new = mergeToOne(X_test, new_feature_test)
    
    
    m2 = xgboost.XGBClassifier(**layer2_param)
    m2.fit(X_train_new2, y_train_2)
    
    
    m3 = xgboost.XGBClassifier(nthread=4)
    m3.fit(X_train, y_train)
    
    print('train set size %s, test set size %s' % (len(train_i), len(test_i)))
    # print('train log_loss %s' % log_loss(y_train, m.predict_proba(X_train)))
    print('combined model log_loss %s\n' % log_loss(y_test, m2.predict_proba(X_test_new)))
    print('single model log_loss %s\n' % log_loss(y_test, m3.predict_proba(X_test)))

    

train set size 78268, test set size 70931
metrics.classification - 1697: 0.095200737911
combined model log_loss 0.095200737911

metrics.classification - 1697: 0.0860219639715
single model log_loss 0.0860219639715

train set size 149199, test set size 68387
metrics.classification - 1697: 0.0905225749597
combined model log_loss 0.0905225749597

metrics.classification - 1697: 0.0850253621848
single model log_loss 0.0850253621848

train set size 217586, test set size 71199
metrics.classification - 1697: 0.0884541214924
combined model log_loss 0.0884541214924

metrics.classification - 1697: 0.0842971726149
single model log_loss 0.0842971726149

train set size 288785, test set size 68318
metrics.classification - 1697: 0.0848797699255
combined model log_loss 0.0848797699255

metrics.classification - 1697: 0.0818784075771
single model log_loss 0.0818784075771

train set size 357103, test set size 63614
metrics.classification - 1697: 0.0799545188852
combined model log_loss 0.0799545188852

metr

# 5 xgboost特征

In [ ]:
# 先用粗糙的树生成特征叶节点
dtsv = DateTimeSplit(dateSeries=offline_df['context_timestamp'], fmt="%Y-%m-%d")

for train_i, test_i in dtsv.split(X, y):
    X_train, y_train = X[train_i], y[train_i]
    X_test, y_test = X[test_i], y[test_i]
    
    

In [89]:
import numpy as np

a = np.array([1,2,3,4,5])
b = np.array([1,1,0,0,1])
c = pd.DataFrame(np.vstack([a,b]).T)
c


,0,1
0,1,1
1,2,1
2,3,0
3,4,0
4,5,1


In [90]:
np.vstack([a,b]).T

array([[1, 1],
       [2, 1],
       [3, 0],
       [4, 0],
       [5, 1]])